<a href="https://colab.research.google.com/github/Felipehonorato1/NaturalLanguageProcessing/blob/main/TransformersFinetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#TRANSFORMERS FINETUNING

---
Using huggingface library


In [21]:
%%capture
# !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz
# !pip install transformers

In [17]:
import os 
from sklearn.model_selection import train_test_split

In [14]:
def read_imdb_data(subset):
  assert subset  in ['train','test']
  labels = []
  sentences = []
  for label in ['pos','neg']:
    for file in os.listdir(f'aclImdb/{subset}/{label}/'):
      f = open(f'aclImdb/{subset}/{label}/{file}')
      sentences.append(f.read())
      if label == 'pos':
        labels.append(1)
      else:
        labels.append(0)

  return sentences,labels


In [26]:
train_sents, train_labels = read_imdb_data('train')
test_sents, test_labels = read_imdb_data('test')

In [38]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_sents, train_labels, test_size = 0.2)

#### Encoding data

In [22]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [39]:
train_encodings = tokenizer(train_texts, truncation = True, padding = True)
val_encodings = tokenizer(val_texts, truncation = True, padding = True)
test_encodings = tokenizer(test_sents, truncation = True, padding = True)

#### Getting TF dataset ready

In [29]:
import tensorflow as tf

In [40]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels))

#### Fine-tuning with Trainer

In [41]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

In [43]:
training_args = TFTrainingArguments(output_dir ='./resultados', num_train_epochs=3, per_device_train_batch_size= 16, per_device_eval_batch_size= 64, warmup_steps= 500)

In [45]:
with training_args.strategy.scope():
  model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [46]:
trainer = TFTrainer(model, args = training_args, train_dataset=train_dataset, eval_dataset= val_dataset)

In [ ]:
trainer.train()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi